# Walmart - Store Sales Forecasting

We are provided with historical sales data for 45 Walmart stores located in different regions, and each store contains many departments. We target to project the sales for each store. To add to the challenge, selected holiday markdown events are included in the dataset. 

There are 45 stores of varying types and sizes.

The data/features:
Store;
Type;
Size;
Dept;
Date;
WeeklySales(training data provided);
IsHoliday;
Temperature - average temperature in the region;
Fuel_price - cost of fuel in the region;
MarkDown1;
MarkDown2;
MarkDown3;
MarkDown4;
MarkDown5;
CPI;
Unemployment

Note: MarkDown data is anonymized data related to promotional markdowns that Walmart is running. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA.

The data covers 2010-02-05 to 2012-11-01.

#### Define Environment

In [1]:
#import all libraries needed

import numpy as np
import pandas as pd
import seaborn as sns
sns.set(style="white")

from patsy import dmatrices, dmatrix
import statsmodels.api as sm

from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import plotly
import plotly.tools as tls
from plotly.offline import plot, iplot
plotly.offline.init_notebook_mode()

import plotly.plotly as py
import plotly.graph_objs as go

import cufflinks as cf
cf.go_offline()

ImportError: No module named plotly

### Data Preparation

#### Import data into pandas dataframe

In [ ]:
df = pd.read_csv("../data/wm_data.csv") #primary data file

# for john's machine: C:\Users\lohnj\Documents\DataBox\DS_HK_9\data\wm_data.csv
# for others' machines: "wm_data.csv"

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
totalsales=df.groupby(['Store'],as_index=False).sum()
avgstoresize=df.groupby(['Store'], as_index=False).mean()

In [ ]:
totalsales['storesize'] = avgstoresize['Size']
totalsales['unit_sales'] = totalsales['Weekly_Sales']/totalsales['storesize'] 
totalsales.drop('Dept',axis=1,inplace=True)
totalsales.drop('Size',axis=1,inplace=True)
totalsales.drop('IsHoliday',axis=1,inplace=True)
totalsales.drop('Temperature',axis=1,inplace=True)
totalsales.drop('Fuel_Price',axis=1,inplace=True)
totalsales.drop('CPI',axis=1,inplace=True)
totalsales.drop('Unemployment',axis=1,inplace=True)
totalsales.drop('MarkDown1',axis=1,inplace=True)
totalsales.drop('MarkDown2',axis=1,inplace=True)
totalsales.drop('MarkDown3',axis=1,inplace=True)
totalsales.drop('MarkDown4',axis=1,inplace=True)
totalsales.drop('MarkDown5',axis=1,inplace=True)

### Initial Data Analysis

#### All store analysis

#### Sales Performance by Store

In [ ]:
trace1 = go.Bar(
    x=totalsales['Store'],
    y=totalsales['Weekly_Sales'],
    name='Store Sales',
    marker=dict(
        color='rgb(253, 194, 13)'
    )
)

data = [trace1]
layout = go.Layout(
    title='Sales by store from 5 Feb 2010 to 1 Nov 2012',
    xaxis=dict(
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Sales',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    legend=dict(
        x=1,
        y=1,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
   
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
    
)
from plotly.offline import iplot
fig = go.Figure(data=data, layout=layout)
plot_url = iplot(fig)

Observation: Store 20 has the highest sales. 

#### Store Size Analysis

In [ ]:
trace1 = go.Bar(
    x=totalsales['Store'],
    y=totalsales['storesize'],
    name='Store Size',
    marker=dict(
        color='rgb(253, 126, 13)'
    )
)

data = [trace1]
layout = go.Layout(
    title='Stores Sizes',
    xaxis=dict(
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Size',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
        
    legend=dict(
        x=1,
        y=1,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
    )
from plotly.offline import iplot
fig = go.Figure(data=data, layout=layout)
plot_url = iplot(fig)

Observation: Store 13 is the largest store. 

#### Sales return per store unit

In [ ]:
trace1 = go.Bar(
    x=totalsales['Store'],
    y=totalsales['unit_sales'],
    name='Sales return per square meter',
    marker=dict(
        color='rgb(55, 83, 109)'
    )
)

data = [trace1]
layout = go.Layout(
    title='Sales return per square meter for each store',
    xaxis=dict(
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Size',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    legend=dict(
        x=1,
        y=1,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)
from plotly.offline import iplot
fig = go.Figure(data=data, layout=layout)
plot_url = iplot(fig)

Observation: Store 10 and Store 43 have highest sales reteurn per store square meter. 

#### Would seasonal factor affect the sales a lot?

In [ ]:
avgsales_nh=df.groupby(['Store','IsHoliday'], as_index=False).mean()

In [ ]:
trace1 = go.Bar(
    x=avgsales_nh['Store'],
    y=avgsales_nh['Weekly_Sales'],
    name='Holiday Sales',
    marker=dict(
        color='rgb(55, 83, 109)'
    )
)
trace2 = go.Bar(
    x=avgstoresize['Store'],
    y=avgstoresize['Weekly_Sales'],
    name='Total Sales',
    marker=dict(
        color='rgb(26, 118, 255)'
    )
)
data = [trace1, trace2]
layout = go.Layout(
    title='Seasonal Sales Contribution',
    xaxis=dict(
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Sales',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    legend=dict(
        x=1,
        y=1,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)
from plotly.offline import iplot
fig = go.Figure(data=data, layout=layout)
plot_url = iplot(fig)

Observation: The average daily sales from Holiday is slightly higher then normal days. The difference is not material.

### Selecting Data

Due to the large dataset, we have opted to look at any given specific store "type", any given store itself, and any given department(s), because otherwise, our machines cannot handle the data. 

In [ ]:
#Constant - can use this to adjust which Store Type to analyse
store_type = 'C'
#Store types are A, B, or C

store_num = 30
#Store numbers 30, 37, 38, 42, 43, 44

dept_num = 1
#department numbers range from 1 to 99

In [ ]:
df.columns = map(str.lower,df.columns)

In [ ]:
dfhelper1 = df[(df['type'] == store_type)]

In [ ]:
dfhelper2 = dfhelper1[(dfhelper1['store'] == store_num)]
#if want to include all stores, then use >= 30

In [ ]:
dfx = dfhelper2[(dfhelper2['dept'] == dept_num)]
#if want to include all stores, then use >= 1

In [ ]:
dfx.head()

#### Simplifying Dataframe

For purposes of EDA, and as noted above, we have set up the analysis to be "controllable" with respect to what types of stores, which stores, and which departments to analyse. To further clean up the dataframe, we can remove the Type and Size because they are essentially one and the same once a particular Store Type is selected (as above).

In [ ]:
#Can remove/drop column 'type' because have established store_type constant above
dfx.drop('type',axis=1,inplace=True)

In [ ]:
#Can also remove/drop column 'size' since already removed 'type' column
dfx.drop('size',axis=1,inplace=True)

In reviewing the data (and included as prompt to the challenge), the mark down data is only available for the last year, so with respect to using them as relevant features for prediction over time, they are not relevant; therefore, we exclude them.

In [ ]:
#Remove markdown columns - not enough data/history of information to be useful as feature
dfx.drop(['markdown1', 'markdown2', 'markdown3', 'markdown4', 'markdown5'], axis=1, inplace=True)

In [ ]:
dfx["date"] = pd.to_datetime(dfx["date"])

In [ ]:
dfx.head()

In [ ]:
dfx.info()

### Single Store Analysis

In [ ]:
#Average sales on non-holidays vs holidays
cols = ['store', 'isholiday', 'weekly_sales']
dfx[cols].groupby(cols[:2]).mean()

#there doesn't seem to be a difference between the average weekly sales for holidays vs non-holidays

On an average basis, there doesn't appear to be a material difference between a week with a holiday or non-holiday weeks.

In [ ]:
#Aggregate sales on non-holidays vs holidays
cols = ['store', 'isholiday', 'weekly_sales']
dfx[cols].groupby(cols[:2]).sum()

In [ ]:
dfx

In [ ]:
dfx.describe()

### Exploratory Data Analysis

In [ ]:
##dfx.iplot(kind='hist', subplots = True, subplot_titles = True, dimensions = (980,1400))

In [ ]:
#In the first instance, to explore/observe any patterns, have chosen to plot for one store and one department
#Ideally, would like to "overlay" the various features of temperature, fuel price, cpi, unemployment over the weekly sales plot
#The subplots are helpful when we only plot one store, and one department; as soon as multiple stores/departments are added,
#the data becomes hard to observe

dfx.iplot(kind = 'scatter', 
         mode = 'markers', 
        # keys = numcols, 
         symbol = 'x', 
         size = 4, 
         x='date', 
         subplots = True, 
         subplot_titles = True, 
         dimensions = (980,1400))

### Observations

1. With respect to the selected store and department, sales are relatively stable over the time period, but there are visibly some periods of higher than average sales.
2. Temperature fluctuates and does not appear to have an obvious relationship with weekly sales.
3. Fuel price trended upward, from 2011; average weekly sales appeared to have a slightly downward trend co-inciding with fuel price increases.
4. As unemployment decreased, CPI increased, which in theory would have suggested weekly sales to average up; however, as noted above, fuel prices also increased during this time, which may or may not have had downward pressure on sales.

### Univariate Analysis and Multivariate Analysis

In [ ]:
dfx.iplot(kind='bar', x='date', y='weekly_sales', dimensions = (1000,400))

Observation: Sales peak were on 11 May 2010 and 11 Apr 2011. 

In [ ]:
dfx.iplot(kind = 'scatter',
          mode = 'markers', 
          symbol = 'x', 
          size = 5, 
          x='date',
          y='weekly_sales',
          dimensions = (800,400))

In [ ]:
#Constant - can use this to adjust which Store Type to analyse
z = 'fuel_price'

#control 'z' as: temperature, fuel_price, cpi, unemployment

In [ ]:
dfx.iplot(kind = 'bubble', 
          x='date',
          y = 'weekly_sales',
          size = z,
          dimensions = (800,400))

### Observations

As mentioned above, the only obvious relationship appears to be between CPI and Unemployment, but neither of these have intuitive relationships with weekly sales over the time period.

## Relationship Exploration

#### Correlationship between the features

In [ ]:
f, ax = plt.subplots(figsize=(12, 12))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.corrplot(df, annot=False, sig_stars=False,
             diag_names=False, cmap=cmap, ax=ax)
f.tight_layout()

### Relationship Between Store Size and Sales

In [ ]:
# explore the types of relationship you should model for, linear?
sns.lmplot("storesize", "Weekly_Sales", totalsales);

In [ ]:
# exploring the types of relationship we should model for, third order polynomial?
sns.lmplot("storesize", "Weekly_Sales", totalsales, order=2);

In [ ]:
sns.lmplot("storesize", "Weekly_Sales", totalsales, order=3);

In [ ]:
# exploring the types of relationship we should model for, fourth order polynomial?
sns.lmplot("storesize", "Weekly_Sales", totalsales, order=4);

In [ ]:
# exploring the types of relationship we should model for, fifth order polynomial?
sns.lmplot("storesize", "Weekly_Sales", totalsales, order=5);

Observation: Forth order and fifth order polynomial looks not good

In [ ]:
sales = totalsales.Weekly_Sales.values
size = totalsales.storesize.values
ys = size

size_poly = dmatrix('C(size, Poly)')

In [ ]:
# Only Size
size_poly1 = dmatrix('size', totalsales)
size_poly1

# Only Size^2
size_sq = dmatrix('np.power(size,2)', totalsales)

# Size + Size^2
size_poly2 = dmatrix('size + np.power(size,2)')

# Orthoganal Polynnomials for ^3, ^5 and ^25
size_poly3 = size_poly[:, :4]
size_poly5 = size_poly[:, :6]
size_poly25 = size_poly[:, :26]

In [ ]:
XNs = [size_poly1, size_sq, size_poly2, size_poly3, size_poly5, size_poly25]

In [ ]:
size_poly1_pred = sm.OLS(ys, size_poly1).fit().predict()
size_sq_pred = sm.OLS(ys, size_sq).fit().predict()
size_poly2_pred = sm.OLS(ys, size_poly2).fit().predict()
size_poly3_pred = sm.OLS(ys, size_poly3).fit().predict()
size_poly5_pred = sm.OLS(ys, size_poly5).fit().predict()
size_poly25_pred = sm.OLS(ys, size_poly25).fit().predict()

In [ ]:
plt.figure()

fig, ax = plt.subplots(3, 2, sharex = True, sharey = True, figsize=(16,9))
fig.subplots_adjust(hspace = 0.0, wspace = 0.0)
fig.suptitle('Polynomial fits to store sales', fontsize = 16.0)

# Iterate through panels (a), model predictions (p), and the polynomial 
# degree of the model (d). Plot the data, the predictions, and label
# the graph to indicate the degree used.
preds = [size_poly1_pred, size_sq_pred, size_poly2_pred, size_poly3_pred, size_poly5_pred, size_poly25_pred]
orders = ['1', 'sq', '2','3', '5', '25']

for a, p, d in zip(ax.ravel(), preds, orders):
    a.plot(size, sales, '.', color = 'steelblue', alpha = 0.5)
    a.plot(size, p, color = 'r')
    a.text(.5, .95, 'D = ' + d, fontsize = 12,
           verticalalignment = 'top',
           horizontalalignment = 'center',
           transform = a.transAxes)
    a.grid()
    
# Alternate axes that have tick labels to avoid overlap.
plt.setp(fig.axes[2].get_yaxis().get_ticklabels(), visible = False)
plt.setp(fig.axes[3].get_yaxis(), ticks_position = 'right')   
plt.setp(fig.axes[1].get_xaxis(), ticks_position = 'top')
plt.setp(fig.axes[3].get_xaxis().get_ticklabels(), visible = False);

In [ ]:
##scores of the models
for order, r in zip(orders, XNs):
    print "%s : %.03f" % (order, sm.OLS(ys, r).fit().rsquared)

The scores look too perfect. Case of overfitting!

## Sales prediction for single store

In [ ]:
dfx.sort('weekly_sales')[['date','temperature','fuel_price','cpi','unemployment','weekly_sales']]

#### Prepare Model

In [ ]:
#Constants
cols = ['temperature', 'fuel_price', 'cpi', 'unemployment']

#cols_a = ['cpi', 'unemployment']

#cols_b = ['temperature', 'fuel_price']

#cols_c = ['fuel_price']

#cols_d = ['temperature']

In [ ]:
#Prepare X and y - build a model for Weekly Sales
X = dfx[cols]
y = dfx['weekly_sales']

### Feature Selection

#### F Values

In [ ]:
from sklearn import feature_selection as fs

def f_regression_feature_selection(input, response):    
# use this against your feature matrix to determine p-values for
# each feature (we care about the second array it returns).
    return fs.univariate_selection.f_regression(input, response)    

In [ ]:
# How many features don't meet the F test threshold?
sum(f_regression_feature_selection(X,y)[0] < 6)

In [ ]:
# Which column are we talking about?
select = f_regression_feature_selection(X,y)[0] < 6
X.columns[select]

In [ ]:
# difference between the ones available and the ones we wish to drop
select = pd.Index(X.columns).difference(pd.Index(X.columns[select]))

Xs = X[select]

In [ ]:
Xs.head()

According to the F Value test, we should build a model using only the variable of temperature (because only temperature passess the threshold test)! However, for purposes of the exercise, we will proceed with evaluating p values. Note: for purposes of analysis, we have arbitrarily reduced the F test threshold above from 10 to 6 so that we can proceed to study all features.

#### P Values

In [ ]:
# How many features don't meet the F test threshold?
sum(f_regression_feature_selection(Xs,y)[1] > 0.05)

This suggests all of the features are significant ( p < 0.05 ) in a univariate regression.

In [ ]:
# Sort the features based on their statistical significance 
ps = f_regression_feature_selection(Xs,y)[1]

p_score = zip(Xs.columns, ps)
ranked_p = sorted(p_score, key=lambda x:x[1])
ranked_p

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
# Build univariate models to see how well each individual feature performs
scores = []
for feat, score in ranked_p:
    est = Ridge()
    X = [[x] for x in Xs[feat]]
    est.fit(X,y)
    scores.append(est.score(X,y))

In [ ]:
# Drop of R^2 with strong to weakest features
plt.plot(range(len(scores)), scores)

All the values below 0.1. None of the features appear to have strong power predicting weekly sales, but relatively speaking, the first feature - temperature stands out.

In [ ]:
# Build models which cummatively look at how well combinations of features perform
scores = []
feats = []
for feat, score in ranked_p:
    est = Ridge()
    feats.append(feat)
    if len(feats) == 1:
        X = [[x] for x in Xs[feat]]
    else:
        X = Xs[feats]
    est.fit(X, y)
    scores.append(est.score(X,y))

In [ ]:
# Drop of R^2 with strong to weakest features
plt.plot(range(len(scores)), scores)

The temperature is the most relevant feature.

In [ ]:
correlation = Xs[[x[0] for x in ranked_p][:]]

In [ ]:
sns.corrplot(correlation)

The fuel_price has strong correlation with CPI

In [ ]:
f, ax = plt.subplots(figsize=(12, 12))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.corrplot(Xs, annot=False, sig_stars=False,
             diag_names=False, cmap=cmap, ax=ax)
f.tight_layout()

Based on the P Values, the most important features are (ranked): temperate, fuel_price, then cpi and umployment. 

In [ ]:
p_features = [x[0] for x in ranked_p][:]
p_features

In [ ]:
# Drop of R^2 with strong to weakest features
plt.plot(range(len(scores)), scores)

As seen above, fuel price has more impact in the improvement in R^2 than cpi or unemployment. Below is feature selection - by manual selection.

#### Feature Selection - Manual

In [ ]:
fs_manual = [ranked_p[x][0] for x in [1,2]]
print fs_manual

In [ ]:
est = Ridge()
X_manual = Xs[fs_manual]
est.fit(X_manual, y).score(X_manual,y)

#### Feature Selection - Forward Selection

In [ ]:
scores = []
feats = []
features = feats[:]
for i in range(len(X.columns)):
    scores = []
    for feat in X.columns.difference(features):
        est = Ridge()
        feats = features[:]
        feats.append(feat)
        Xs = X[feats]
        #print feats
        est.fit(Xs, y)
        feats = features
        scores.append([feat,est.score(Xs,y)])
    dfx = pd.DataFrame(scores, columns=['a','s'])
    dfx.index = dfx.a
    #print dfx
    addfeat = dfx.s.argmax()
    print ('Added feature: ' + addfeat + ', new score is: ' + str(dfx.s.max()))
    features.append(addfeat)

In [ ]:
fs_fwd = features[:]

In [ ]:
print fs_fwd

#### Feature Selection - RFE

In [ ]:
stand_X = (X - X.mean()) / X.std()

In [ ]:
stand_X.columns

In [ ]:
from sklearn.feature_selection import RFE

# Create the RFE object and rank each feature
est = Ridge(alpha = 100)
rfe = RFE(estimator=est, n_features_to_select=1, step=1)

rfe.fit(stand_X, y)
ranking = rfe.ranking_
            
scores = zip(Xs.columns,ranking)
scores = sorted(scores, key=lambda x:x[1])
fs_rfe = [x[0] for x in scores][:]

In [ ]:
plt.figure(figsize=(16, 6))
sns.barplot(Xs.columns, 16 - ranking);

From the above chart, the priority of the important features are temperature, unemployment, CPI then fuel_price.

In [ ]:
print 'Feature Selection - Manual:', fs_manual
print 'Feature Selection - Forward Selection (p-value):', fs_fwd
print 'Feature Selection - RFE:', fs_rfe

In [ ]:
for feats in [fs_manual, fs_fwd, fs_rfe]:
    est = Ridge()
    Xs = X[feats]
    print est.fit(Xs, y).score(Xs,y)

Feature selection via forward selection or RFE is better than handpicking, but overall, neither appears to be a strong model!!

In [ ]:
Xs = stand_X[fs_rfe]
Xs = sm.add_constant(Xs)

In [ ]:
regress = sm.OLS(y,Xs)
results = regress.fit()

In [ ]:
print results.summary()

### Conclusion
The model scores are not very good for prediction. Therefore we focus on determining the impact of the features to Walmart sales. From the analysis, temperature, CPI, unemployment and Fuel_price have impact to the sales and the temperature has the greatest impact.